# Perform NER on Book 4 (BIO format)

The notebook contains the code to perform Flair NER and spaCy NER on Book 4 and structure the output in the BIO style format.

We used two different NER systems:

- Flair (https://flairnlp.github.io/docs/tutorial-basics/tagging-entities)
- SpaCy (https://spacy.io/models/en)

Each system was tested according to different parameters. More specifically, we compared:

- Flair ner
- Flair ner-large
- Flair ner-large in combination with SegtokSentenceSplitter
- en_core_web_md
- en_core_web_trf

The resulting .csv file contains a list of all the tokens in Book 4 (18,664) including punctuation and special characters. Each token is associated to its reference position (book, chapter, paragraph), the position of the token in the paragraph ('Index' column), the start position of the token in the paragraph at the character level ('Start pos' column), the BIO annotation in Flair, Flair-large and Flair+Splitter with the precision score and the BIO annotation in spaCy.

For an introduction to the principles of the BIO style annotation see: https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup

## import Flair
from flair.data import Sentence
from flair.nn import Classifier
from flair.models import SequenceTagger
from flair.splitter import SegtokSentenceSplitter

## import spaCy
import spacy

In [ ]:
## open the source HTML page as soup by BeautifulSoup
soup = BeautifulSoup(open("/Users/u0154817/OneDrive - KU Leuven/Documents/KU Leuven/PhD project 'Greek Spaces in Roman Times'/Data_Extraction/Sources/NH_Eng_ToposText/NH_Eng_1-11.html", encoding='utf-8'), features="lxml")

## get all the paragraphs in Book 4
book_4 = soup.find_all("p", id=lambda x: x and x.startswith("urn:cts:latinLit:phi0978.phi001:4.")) ## get all the paragraph starting with the ID phi0978.phi001:4.

In [ ]:
reference_column = []
index_column = []
token_column = []
start_pos_column = []
BIO_column = []
BIO_precision = []

# Flair ner

In [ ]:
for paragraph in book_4: ## for each paragraph in Book 4
    
    p_tag_id = paragraph['id'] ## get the id of the paragraph
    print(p_tag_id) ## print the id of the paragraph
    text = paragraph.get_text() ## get the text of the paragraph
    
    ## create temporary lists
    reference_column_temp = []
    index_column_temp = []
    token_column_temp = []
    start_pos_column_temp = []
    BIO_column_temp = []
    BIO_precision_temp = []

    ## make a Flair Sentence from the text of the paragraph using the Flair Sentence function
    sentence = Sentence(text)

    ## load the NER tagger ner
    tagger = Classifier.load('ner')

    ## run NER over the Sentence
    tagger.predict(sentence)

    for index, token in enumerate(sentence): ## for each token in the Sentence
        
        token_text = token.text ## get the text of the token
        start_pos = token.start_position ## get the start position of the token

        reference_column_temp.append(p_tag_id) ## append the id of the paragraph
        index_column_temp.append(index) ## append the index of the token
        token_column_temp.append(token_text) ## append the token
        start_pos_column_temp.append(start_pos) ## append the start position of the token
        BIO_column_temp.append('O') ## append a temporary O label
        BIO_precision_temp.append('-') ## append a temporary - value
        
    
    for entity in sentence.get_spans('ner'): ## for each NER Entity in the Sentence
        
        entity_label = entity.labels[0].value ## extract the label assigned to the Entity
        entity_score = entity.labels[0].score ## get the probability score for the label
    
        for index,token in enumerate(entity): ## for each token in the Entity
            start_pos_token = token.start_position ## get the start position of the token
        
            if index == 0: ## if it is the first token in the Entity
                entity_label_token = 'B-'+str(entity_label) ## the label starts with B-(eginning)
            else : ## if not
                entity_label_token = 'I-'+str(entity_label) ## the label starts with I-(nside)
                    
            for i,start_position in enumerate(start_pos_column_temp): ## for each start position in the list created above
                
                if int(start_pos_token) == int(start_position): ## find the correspondent position of the token
                    BIO_column_temp[i] = entity_label_token ## update the BIO column
                    BIO_precision_temp[i] = entity_score ## update the Precision column
        
    reference_column.extend(reference_column_temp)
    index_column.extend(index_column_temp)
    token_column.extend(token_column_temp)
    start_pos_column.extend(start_pos_column_temp)
    BIO_column.extend(BIO_column_temp)
    BIO_precision.extend(BIO_precision_temp)

In [ ]:
## create a pandas dataframe

data = {
    'Reference': reference_column,
    'Index': index_column,
    'Token': token_column,
    'Start_pos': start_pos_column,
    'BIO_Flair': BIO_column,
    'Precision_Flair': BIO_precision
}

df = pd.DataFrame(data)

# Flair ner-large

In [ ]:
BIO_Flair_large_column = []
Precision_Flair_large_column = []

In [ ]:
for paragraph in book_4: 
    
    p_tag_id = paragraph['id'] 
    print(p_tag_id) 
    text = paragraph.get_text() 

    start_pos_column_temp = []
    BIO_Flair_large_column_temp = []
    Precision_Flair_large_column_temp = []
    
    sentence = Sentence(text)

    ## load the NER tagger ner-large
    tagger = Classifier.load('ner-large')

    tagger.predict(sentence)
    
    for index, token in enumerate(sentence):
        
        start_pos = token.start_position 

        start_pos_column_temp.append(start_pos)
        BIO_Flair_large_column_temp.append('O')
        Precision_Flair_large_column_temp.append('-')

    for entity in sentence.get_spans('ner'):
    
        entity_label = entity.labels[0].value 
        entity_score = entity.labels[0].score
    
        for index,token in enumerate(entity):
            start_pos_token = token.start_position
        
            if index == 0: 
                entity_label_token = 'B-'+str(entity_label)
            else :
                entity_label_token = 'I-'+str(entity_label) 
                    
            for i,start_position in enumerate(start_pos_column_temp):
            
                if int(start_pos_token) == int(start_position):
                    BIO_Flair_large_column_temp[i] = entity_label_token
                    Precision_Flair_large_column_temp[i] = entity_score
    
    BIO_Flair_large_column.extend(BIO_Flair_large_column_temp)
    Precision_Flair_large_column.extend(Precision_Flair_large_column_temp)

In [ ]:
df['BIO_Flair-large'] = BIO_Flair_large_column
df['Precision_Flair-large'] = Precision_Flair_large_column

# Flair ner-large + SegtokSentenceSplitter

In [ ]:
## initialize sentence splitter
splitter = SegtokSentenceSplitter()

In [ ]:
BIO_Flair_splitter_column = []
Precision_Flair_splitter_column = []

In [ ]:
for paragraph in book_4: ## for each paragraph in Book 4
    
    p_tag_id = paragraph['id'] ## get the id of the paragraph
    print(p_tag_id) ## print the id of the paragraph
    text = paragraph.get_text() ## get the text of the paragraph

    ## use Splitter to split the text into a list of Sentences
    sentences = splitter.split(text)
    
    ## load the NER tagger ner-large
    tagger = Classifier.load('ner-large')
    tagger.predict(sentences)

    for index,sentence in enumerate(sentences): ## for each Sentence in Sentences
        
        if len(sentence) > 0: ## if the Sentence exists
                    
            start_pos_splitter_column = []
            BIO_Flair_splitter_column_temp = []
            Precision_Flair_splitter_temp = []
        
            for token in sentence: ## for each token in the splitted Sentence
                
                start_pos_splitter = str(token.start_position) 
            
                start_pos_splitter_column.append(start_pos_splitter) 
                BIO_Flair_splitter_column_temp.append('O') 
                Precision_Flair_splitter_temp.append('-') 
                        
            for entity in sentence.get_spans('ner'): ## for each NER Entity in the splitted Sentence
        
                entity_label = entity.labels[0].value 
                entity_score = entity.labels[0].score 
    
                for index1,token in enumerate(entity):
            
                    start_pos_token = str(token.start_position) 
        
                    if index1 == 0: 
                        entity_label_token = 'B-'+str(entity_label) 
                    else :
                        entity_label_token = 'I-'+str(entity_label)
                    
                    for index2,start_position in enumerate(start_pos_splitter_column):
            
                        if str(start_pos_token) == str(start_position):
                            BIO_Flair_splitter_column_temp[index2] = entity_label_token
                            Precision_Flair_splitter_temp[index2] = entity_score
    
            BIO_Flair_splitter_column.extend(BIO_Flair_splitter_column_temp)
            Precision_Flair_splitter_column.extend(Precision_Flair_splitter_temp)

In [ ]:
df['BIO_Flair_Splitter'] = BIO_Flair_splitter_column
df['Precision_Flair_Splitter'] = Precision_Flair_splitter_column

# spaCy-md

In [ ]:
df['BIO_spaCy-md'] = 'O' ## create a column with temporary O label

In [ ]:
## load the NER model
nlp_spaCy = spacy.load("en_core_web_md")

We observed that the nlp_spaCy tokenization is slighlty different from the Flair tokenization and the list of tokens obtained from spaCy does not align with the dataframe.

In [ ]:
for paragraph in book_4: ## for each paragraph in Book 4
    
    p_tag_id = paragraph['id'] ## get the id of the paragraph
    print(p_tag_id) ## print the id of the paragraph
    text = paragraph.get_text() ## get the text of the paragraph

    processed_text = nlp_spaCy(text) ## run NER over the paragraph
    
    for token in processed_text: ## for each token
        
        if token.ent_type_: ## if the token has an Entity label
            
            token = entity ## the token is an Entity
            
            entity_label_token = str(entity.ent_iob_)+'-'+entity.ent_type_ ## extract the label assigned to the Entity
            start_pos_token = entity.idx ## get the start position of the Entity
            paragraph_start_pos = str(p_tag_id)+'.'+str(start_pos_token) ## get the reference+start position
    
            for i, reference in enumerate(df['Reference']): ## for each token in the dataframe
                
                reference_position = str(reference)+'.'+str(df['Start_pos'][i]) ## get the reference+start position
                if paragraph_start_pos == reference_position: ## if it is the same
                    
                    df['BIO_spaCy-md'][i] = entity_label_token ## update the label column

# spaCy-trf

In [ ]:
df['BIO_spaCy-trf'] = 'O'

In [ ]:
## load the NER model
nlp_spaCy = spacy.load("en_core_web_trf")

In [ ]:
for paragraph in book_4:
    
    p_tag_id = paragraph['id']
    print(p_tag_id)
    text = paragraph.get_text()

    processed_text = nlp_spaCy(text)
    
    for token in processed_text: 
        
        if token.ent_type_:
            
            token = entity
            
            entity_label_token = str(entity.ent_iob_)+'-'+entity.ent_type_
            start_pos_token = entity.idx 
            paragraph_start_pos = str(p_tag_id)+'.'+str(start_pos_token)
    
            for i, reference in enumerate(df['Reference']):
                
                reference_position = str(reference)+'.'+str(df['Start_pos'][i])
                if paragraph_start_pos == reference_position:
                    
                    df['BIO_spaCy-trf'][i] = entity_label_token

In [ ]:
df.to_csv("BIO_NER_Flair_spaCy.csv")